In [1]:
%pip install leafmap
%pip install rasterio
%pip install localtileserver # Need to visualize a rather large  raster?
%pip install segment-geospatial
%pip install rioxarray

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.9/182.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.7 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [2]:
import leafmap
import rasterio
import os
from samgeo import SamGeo, tms_to_geotiff
import datetime
from google.colab import files

In [3]:
url =  "https://drive.google.com/file/d/1Sbwjjlx9nizlDuU6xdTb1hvCwtUU4_LM/view?usp=share_link"
gpkg= "ortho_20240605_20240613_JDT4_RGB_8bit_clip_7" +"_annotations.gpkg"
gpkg_path = '/content/'+gpkg
leafmap.download_file(url, output="image.tif", overwrite=True)

Downloading...
From: https://drive.google.com/uc?id=1Sbwjjlx9nizlDuU6xdTb1hvCwtUU4_LM
To: /content/image.tif
100%|██████████| 7.84M/7.84M [00:00<00:00, 15.6MB/s]


'/content/image.tif'

In [4]:
sam_kwargs = {
    "points_per_side": 128,
    "pred_iou_thresh": 0.80,
    "stability_score_thresh": 0.85,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 10,
}

sam = SamGeo(
    model_type="vit_l",
 #   model_type="vit_h",
    #model_type="vit_b",
    sam_kwargs=sam_kwargs,
)

Model checkpoint for vit_l not found.


Downloading...
From: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth
To: /root/.cache/torch/hub/checkpoints/sam_vit_l_0b3195.pth
100%|██████████| 1.25G/1.25G [00:09<00:00, 129MB/s] 


In [ ]:
current_time = datetime.datetime.now()

sam.generate("image.tif", output="mask.tif", foreground= False #True
             )
#sam.generate("image.tif", output="mask.tif", unique=True, foreground=True, batch=True, batch_sample_size= (576, 576) )
end_time = datetime.datetime.now()
print(   current_time.hour- end_time.hour )
print(   current_time.minute- end_time.minute )

sam.show_anns(axis="off", alpha=1, output="annotations.tif")

sam.tiff_to_vector( "annotations.tif",  gpkg )

files.download(gpkg_path)